In [ ]:
# 1. Setup y carga librerías
import os, pandas as pd, geopandas as gpd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
from pathlib import Path
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()
sns.set_palette('viridis')
POSTGRES_USER=os.getenv('POSTGRES_USER'); POSTGRES_PASSWORD=os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST=os.getenv('POSTGRES_HOST','localhost'); POSTGRES_PORT=os.getenv('POSTGRES_PORT','5432'); POSTGRES_DB=os.getenv('POSTGRES_DB')
engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')
print('Conexión lista')

In [ ]:
# 2. Carga de datasets clave
def load_gdf(schema, table):
    try:
        return gpd.read_postgis(f'SELECT * FROM {schema}.{table}', engine, geom_col='geometry')
    except Exception as e:
        print(f'Error cargando {schema}.{table}:', e)
        return None
manz_attr = load_gdf('processed_data','manzanas_atributos')
uso = load_gdf('processed_data','manzanas_uso_suelo')
net_nodes = load_gdf('processed_data','network_nodes_metrics')
# Métricas tabulares (sin geometría)
metrics = pd.read_sql('SELECT * FROM processed_data.metrics_manzanas', engine)
print('Shapes:', len(manz_attr) if manz_attr is not None else None, len(metrics))

In [ ]:
# 3. Unificación de atributos clave por manzana
# Selección mínima de columnas relevantes para exploración
merged = metrics.copy()
# Asegurar alias clave
key = 'manzent' if 'manzent' in merged.columns else ('MANZENT' if 'MANZENT' in merged.columns else merged.columns[0])
if manz_attr is not None:
    geo_subset = manz_attr[[c for c in manz_attr.columns if c in [key,'geometry']]]
    geo_subset[key] = geo_subset[key].astype(str)
    merged[key] = merged[key].astype(str)
    merged_geo = geo_subset.merge(merged, on=key, how='left')
else:
    merged_geo = None
print('Columnas merged:', merged.columns.tolist()[:15])

In [ ]:
# 4. Estadísticas descriptivas
desc_cols = [c for c in merged.columns if merged[c].dtype!='object' and c not in ['geometry']]
stats = merged[desc_cols].describe().T
stats[['mean','std','min','max']].head(10)

In [ ]:
# 5. Correlaciones básicas entre variables numéricas seleccionadas
sel = [col for col in ['edificios_count','amenidades_count','area_m2','ndvi_mean','road_length_m','road_density_m_per_km2'] if col in merged.columns]
corr = merged[sel].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap='magma', fmt='.2f')
plt.title('Matriz de correlación variables clave')
plt.tight_layout()
plt.show()

In [ ]:
# 6. Mapas temáticos: NDVI y densidad vial
if merged_geo is not None:
    fig, axes = plt.subplots(1,2, figsize=(12,6))
    if 'ndvi_mean' in merged_geo.columns:
        merged_geo.plot(column='ndvi_mean', ax=axes[0], cmap='YlGn', legend=True, missing_kwds={'color':'lightgrey'})
        axes[0].set_title('NDVI medio por manzana')
    if 'road_density_m_per_km2' in merged_geo.columns:
        merged_geo.plot(column='road_density_m_per_km2', ax=axes[1], cmap='OrRd', legend=True, missing_kwds={'color':'lightgrey'})
        axes[1].set_title('Densidad vial (m/km²)')
    for ax in axes: ax.axis('off')
    plt.tight_layout(); plt.show()
else:
    print('No hay geometría de manzanas para mapear.')

In [ ]:
# 7. Distribución de NDVI y edificios (histogramas)
plot_vars = [v for v in ['ndvi_mean','edificios_count'] if v in merged.columns]
for v in plot_vars:
    plt.figure(figsize=(5,3))
    sns.histplot(merged[v], kde=True, bins=30, color='steelblue')
    plt.title(f'Distribución {v}')
    plt.tight_layout()
    plt.show()

In [ ]:
# 8. Autocorrelación espacial (Moran Global / Local)
# Excluye la manzana huérfana si está presente
try:
    import pysal
    from pysal.lib import weights
    from pysal.explore import esda
    import splot.esda as splot_esda
    import numpy as np
    import matplotlib.pyplot as plt
    if merged_geo is not None:
        gdf_valid = merged_geo[merged_geo.geometry.notnull()].copy()
        if 'manzent' in gdf_valid.columns:
            gdf_valid = gdf_valid[gdf_valid['manzent'] != '13129991999999']
        gdf_valid = gdf_valid[gdf_valid.geometry.is_valid]
        # Seleccionar variable objetivo principal para Moran (prioriza ndvi_mean)
        target_var = 'ndvi_mean' if 'ndvi_mean' in gdf_valid.columns else [c for c in gdf_valid.columns if c not in ['geometry','manzent','MANZENT'] and gdf_valid[c].dtype != 'object'][0]
        w = weights.Queen.from_dataframe(gdf_valid)
        w.transform = 'r'
        y = gdf_valid[target_var].fillna(gdf_valid[target_var].mean())
        mi = esda.Moran(y, w)
        lisa = esda.Moran_Local(y, w)
        print(f"Moran's I ({target_var}): {mi.I:.4f}")
        print(f"P-value (norm): {mi.p_norm:.4f}")
        # Scatterplot Moran y mapa clusters
        fig, axes = plt.subplots(1, 2, figsize=(14,6))
        splot_esda.moran_scatterplot(mi, ax=axes[0])
        axes[0].set_title('Moran Scatterplot')
        # Clasificación de clusters (HH, LL, HL, LH) según cuadrante y significancia
        sig = lisa.p_sim < 0.05
        quad = lisa.q
        cluster_labels = np.where(sig & (quad==1), 'Alto-Alto',
                           np.where(sig & (quad==2), 'Bajo-Alto',
                           np.where(sig & (quad==3), 'Bajo-Bajo',
                           np.where(sig & (quad==4), 'Alto-Bajo','No Sig.'))))
        gdf_valid['lisa_cluster'] = cluster_labels
        cmap = {'Alto-Alto':'#d7191c','Bajo-Bajo':'#2c7bb6','Alto-Bajo':'#fdae61','Bajo-Alto':'#abd9e9','No Sig.':'#cccccc'}
        gdf_valid.plot(column='lisa_cluster', ax=axes[1], color=gdf_valid['lisa_cluster'].map(cmap))
        axes[1].set_title(f'Clusters LISA ({target_var})')
        for ax in axes: ax.axis('off')
        plt.tight_layout(); plt.show()
    else:
        print('Geometría no disponible para análisis espacial.')
except Exception as e:
    print('Error en cálculo Moran/LISA. Verifique dependencias pysal/splot:', e)

In [ ]:
# 8E. PCA espacial multivariado
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
if merged_geo is not None:
    # Selección de variables numéricas para PCA (excluye identificadores y geometry)
    pca_vars = [v for v in numeric_cols if v not in ['manzent','MANZENT'] and merged[v].notnull().sum() > 0]
    df_pca = merged[pca_vars].fillna(merged[pca_vars].median())
    scaler = StandardScaler()
    X = scaler.fit_transform(df_pca)
    pca = PCA(n_components=2)
    comps = pca.fit_transform(X)
    merged_geo['PC1'] = comps[:,0]
    merged_geo['PC2'] = comps[:,1]
    print('Varianza explicada PC1:', round(pca.explained_variance_ratio_[0],3))
    print('Varianza explicada PC2:', round(pca.explained_variance_ratio_[1],3))
    # Mapear PC1 y PC2
    for pc_col in ['PC1','PC2']:
        plot_choropleth(merged_geo, pc_col, scheme='Quantiles', k=5, cmap='plasma', title=f'{pc_col} (PCA)')
else:
    print('No se puede ejecutar PCA: geometría o variables faltantes.')

In [ ]:
# 8D. Hotspots & clusters (export resultados)
if merged_geo is not None and 'lisa_cluster' in merged_geo.columns:
    out_clusters = Path('../data/processed/lisa_clusters.geojson')
    try:
        merged_geo.to_file(out_clusters, driver='GeoJSON')
        print('Exportado LISA clusters en', out_clusters)
    except Exception as e:
        print('Error exportando clusters:', e)
else:
    print('No se encontraron resultados LISA para exportar.')

In [ ]:
# 8C. Generar al menos 10 mapas temáticos
maps_target = []
priority_vars = ['ndvi_mean','edificios_count','amenidades_count','road_density_m_per_km2','road_length_m','area_m2']
for v in priority_vars:
    if v in merged_geo.columns: maps_target.append(v)
# Completar hasta 10 con otras numéricas
for v in numeric_cols:
    if len(maps_target) >= 10: break
    if v not in maps_target and v in merged_geo.columns and v not in ['geometry','manzent','MANZENT']:
        maps_target.append(v)
print('Mapeando variables:', maps_target)
for v in maps_target:
    plot_choropleth(merged_geo, v, scheme='Quantiles', k=5, cmap='viridis', title=f'Mapa {v}')

In [ ]:
# 8B. Función para mapas temáticos profesionales
import matplotlib.colors as mcolors
import mapclassify
def plot_choropleth(gdf, column, scheme='Quantiles', k=5, cmap='viridis', title=None):
    try:
        gdf_plot = gdf[gdf[column].notnull()].copy()
        classifier = mapclassify.Quantiles(gdf_plot[column], k=k) if scheme=='Quantiles' else mapclassify.NaturalBreaks(gdf_plot[column], k=k)
        gdf_plot['cls'] = classifier.yb
        fig, ax = plt.subplots(figsize=(6,6))
        gdf_plot.plot(column='cls', ax=ax, cmap=cmap, legend=True,
                      legend_kwds={'label': f'{column} ({scheme})','orientation':'vertical'})
        ax.set_title(title or column)
        ax.axis('off')
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f'Error graficando {column}:', e)
available_vars = [v for v in numeric_cols if v not in ['geometry']][:20]
print('Variables disponibles para mapas (primeras 20):', available_vars)

In [ ]:
# 8A. Estadísticas descriptivas espaciales extendidas
numeric_cols = [c for c in merged.columns if merged[c].dtype != 'object' and c not in ['geometry']]
extended_stats = merged[numeric_cols].describe(percentiles=[0.05,0.25,0.5,0.75,0.95]).T
extended_stats.head(15)

### Nota Moran/LISA
Para análisis completo: separar en celdas dedicadas, crear scatterplot de Moran y mapa LISA (splot). Esto es solo verificación preliminar.

In [ ]:
# 9. Guardar subset para futuros análisis
out_path = Path('../data/processed/esda_subset.geojson')
if merged_geo is not None:
    try:
        cols_keep = [c for c in merged_geo.columns if c in ['manzent','ndvi_mean','edificios_count','amenidades_count','road_density_m_per_km2']] + ['geometry']
        subset = merged_geo[cols_keep]
        subset.to_file(out_path, driver='GeoJSON')
        print('Guardado subset ESDA en', out_path)
    except Exception as e:
        print('Error guardando subset ESDA:', e)

## Checklist Exploratorio (Actualizado)
- [x] Datasets cargados
- [x] Estadísticas descriptivas básicas
- [x] Estadísticas espaciales extendidas (percentiles, variables numéricas)
- [x] Mapas temáticos (>=10 variables)
- [x] Moran Global y Local (LISA) con clusters
- [x] Export clusters LISA
- [x] PCA espacial con primeras dos componentes mapeadas

## Próximos pasos ESDA
1. Añadir análisis bivariado de autocorrelación (Moran Bivariado si aplica).
2. Incluir mapas de significancia LISA separados y boxplots por cluster.
3. Calcular indicadores per cápita (requiere población microdatos agregada).
4. Explorar Geary's C y Getis-Ord G* para hotspots alternativos.
5. Integrar land use proportions si se generan (uso_suelo_unificado).
